In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
print("TF:", tf.__version__)
print("GPUs visíveis:", tf.config.list_physical_devices('GPU'))


TF: 2.19.0
GPUs visíveis: []


In [3]:
import os
import zipfile

if not os.path.exists('/content/trashnet'):
    print("Baixando o repositório do TrashNet...")
    !git clone https://github.com/garythung/trashnet.git
else:
    print("Repositório já baixado.")

zip_path = "/content/trashnet/data/dataset-resized.zip"
extract_path = "/content/dataset_final"

if os.path.exists(zip_path):
    print(f"Descompactando {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
    print("Descompactação concluída!")
else:
    print("Erro: Arquivo zip não encontrado no repositório.")

Baixando o repositório do TrashNet...
Cloning into 'trashnet'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 45 (delta 6), reused 0 (delta 0), pack-reused 33 (from 1)
Receiving objects: 100% (45/45), 40.64 MiB | 39.82 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Descompactando /content/trashnet/data/dataset-resized.zip...
Descompactação concluída!


In [ ]:
import pydot
print("Pydot version:", pydot.__version__)
import shutil
print("Graphviz no PATH:", shutil.which("dot"))



In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.utils as utils
from   tensorflow.keras.preprocessing.image import ImageDataGenerator
from   tensorflow.keras.models import Sequential
from   tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, ZeroPadding2D
from   tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from   tensorflow.keras.utils import plot_model
from   tensorflow.keras.callbacks import ModelCheckpoint
from   pathlib import Path
from   tensorflow.keras import models, layers, optimizers
from   sklearn.utils.class_weight import compute_class_weight

In [ ]:
!pip install -q datasets
from datasets import load_dataset
import os
import shutil
from tqdm import tqdm

dataset = load_dataset("garythung/trashnet", split="train")
print(dataset)

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path
import os


possible_paths = [
    Path("/content/dataset_final/dataset-resized"),
    Path("/content/trashnet/data/dataset-resized"),
    Path("/content/dataset-resized")
]

data_path = None
for p in possible_paths:
    if p.exists():
        data_path = p
        break


if data_path:
    print(f"✅ Pasta de imagens encontrada em: {data_path}")
    print("Classes encontradas:", [d.name for d in data_path.iterdir() if d.is_dir()])
else:
    print("❌ Erro: A pasta do dataset não foi encontrada. Rode a célula de download novamente.")


In [ ]:
from pathlib import Path
import pandas as pd

data_path = Path("/content/dataset_final/dataset-resized")
print("Classes encontradas:", [d.name for d in data_path.iterdir() if d.is_dir()])

data = []
for folder in data_path.iterdir():
    if folder.is_dir():
        file_count = len(list(folder.glob("*")))
        data.append({"Folder Name": folder.name, "File Count": file_count})

count = pd.DataFrame(data).set_index("Folder Name")
count


In [ ]:
print(f'Total {count.sum()}')

In [ ]:

def plot_imgs(item_dir, top=10):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][:5]

    plt.figure(figsize=(10, 10))

    for idx, img_path in enumerate(item_files):
        plt.subplot(5, 5, idx+1)

        img = plt.imread(img_path)
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.title(os.path.basename(item_dir))

In [ ]:
plot_imgs(data_path / 'cardboard')
plot_imgs(data_path / 'glass')
plot_imgs(data_path / 'metal')
plot_imgs(data_path / 'paper')
plot_imgs(data_path / 'plastic')
plot_imgs(data_path / 'trash')


In [ ]:
# Define variable for requirements
batch_size = 30
target_size = (128, 128)

validation_split = 0.15

In [ ]:
train_img_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.5,
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=50,
    shear_range=0.3,
    fill_mode='nearest'
)

test_data_img_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split
)


In [ ]:
train_data = train_img_generator.flow_from_directory(
    data_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='training',
    shuffle=True,
    seed=42
)

test_data = test_data_img_generator.flow_from_directory(
    data_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation',
    shuffle=False,
    seed=42
)


In [ ]:
# Separate train Data
train_data = train_img_generator.flow_from_directory(data_path,
                                                        subset="training",
                                                        seed=133,
                                                        target_size=target_size,
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        color_mode='rgb',
                                                       )

In [ ]:
# Separate validation Data
test_data = test_data_img_generator.flow_from_directory(data_path,
                                                        subset="validation",
                                                        seed=133,
                                                        target_size=target_size,
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        color_mode='rgb',
                                                       )

In [ ]:
model = Sequential()
model.add(ZeroPadding2D(padding=(1, 1), input_shape=(target_size[0], target_size[1], 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.4))
model.add(MaxPool2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
import sys
print("Python do notebook:", sys.executable)

!{sys.executable} -m pip install pydot graphviz

import pydot
print("Pydot carregado de:", pydot.__file__)


In [ ]:
import shutil
print("dot no sistema:", shutil.which("dot"))


In [ ]:
model.summary()


In [ ]:
ES = EarlyStopping(monitor='val_loss', mode='min', patience=10)

In [ ]:
RLROP = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1, min_lr=0.01)

In [ ]:
MCH = ModelCheckpoint('trash.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()]
             )

In [ ]:
print("Classes e índices:", train_data.class_indices)
print("Número de classes:", train_data.num_classes)


In [ ]:
epochs = 20

history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=test_data,
    callbacks=[ES, RLROP, MCH]
)


In [ ]:

loss, acc = model.evaluate(test_data)
print(f"Loss de validação: {loss:.4f}")
print(f"Acurácia de validação: {acc:.4f}")


In [ ]:
best_score = max(history.history['val_categorical_accuracy'] )
print(f"Best Validation score is: {best_score}")

In [ ]:
# Accuracy on train data :
accuracy = history.history['categorical_accuracy']

# Accuracy on test data :
val_accuracy = history.history['val_categorical_accuracy']
epochs = range(1, len(accuracy) + 1)  # Epochs

# Accuracy line plot :
plt.figure(figsize=(10, 5))
plt.plot(epochs, accuracy, 'b', label='train accuracy')
plt.plot(epochs, val_accuracy, 'r', label='test accuracy')
plt.title('Accuracy plot')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend();

In [ ]:
classes = train_data.class_indices
classes

In [ ]:
import os

folder = "/content/dataset_final/dataset-resized/paper"
print(os.listdir(folder)[:20])


In [ ]:
from tensorflow.keras.preprocessing import image as utils

img_path = "/content/dataset_final/dataset-resized/paper/paper11.jpg"

img = utils.load_img(
    img_path,
    color_mode='rgb',
    target_size=target_size,
    interpolation='nearest'
)

img


In [ ]:

import numpy as np
import tensorflow as tf

img_array = utils.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)

idx_to_class = {v: k for k, v in train_data.class_indices.items()}

pred_class_idx = np.argmax(pred, axis=1)[0]
print("Classe prevista:", idx_to_class[pred_class_idx])
print("Distribuição de probabilidade:", pred[0])


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

test_data.shuffle = False

y_true = test_data.classes
y_pred_proba = model.predict(test_data)
y_pred = np.argmax(y_pred_proba, axis=1)

cm = confusion_matrix(y_true, y_pred)
labels = list(test_data.class_indices.keys())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de confusão")
plt.show()

print(classification_report(y_true, y_pred, target_names=labels))
